In [1]:
import pyodbc
import pandas as pd
import os
import numpy as np

# Thông tin kết nối đến SQL Server
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=10.16.22.30;'
    'DATABASE=Thanglong_2024(IBOSS);'
    'UID=foxai;'
    'PWD=123'
)

# Danh sách các bảng cần lấy dữ liệu
queries = {
    "FOXAI_Baocao_BangCDKT":
    """ SELECT 
            Ngay [Ngày], 
            Maso [ID], 
            Tenchitieu [Khoản mục],
            Socuoiky [Giá trị]
        FROM FOXAI_Baocao_BangCDKT
        ORDER BY Ngay ASC;
    """,
    "FOXAI_Baocao_KetquaKinhdoanh_PI":
    """ SELECT
            Ngay [Ngày],
            Maso [ID],
            Tenchitieu [Khoản mục],
            Kynay [Giá trị]
        FROM FOXAI_Baocao_KetquaKinhdoanh_PI
        ORDER BY Ngay ASC;
    """,
    "FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B6":
    """ SELECT
        TuNgay [Ngày],
        C1 [Mã NL],
        C2 [Tên NL],
        C5 [Mã TP],
        C6 [Tên TP],
        C7 [SL TP Nhập Kho],
        C3 [SL NL SX],
        C4 [Thành Tiền],
        C9 [SL NL Tính Vào GT],
        C10 [Thành Tiền],
        C11 [Tiêu Hao NL (kg/1000b)]
        FROM FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B6
        ORDER BY TuNgay ASC;
    """,
    "FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B12":
    """SELECT
        TuNgay [Ngày],
        C1 [Mã TP],
        C2 [Tên SP],
        C3 [SL TP Nhập Kho],
        C4 [SL (NVL Chính)],
        C5 [Tiền (NVL Chính)],
        C6 [T.Hao (NVL Chính)],
        C7 [Đ.Giá (NVL Chính)],
        C8 [Tiền (VT-PL)],
        C9 [Đ.Giá (VT-PL)],
        C10 [Tiền (NCTT)],
        C11 [Đ.Giá (NCTT)],
        C12 [Tiền (SXC)],
        C13 [Đ.Giá (SXC)],
        C14 [Tiền (Total)],
        C15 [Đ.Giá (Total)]
        FROM FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B12
        ORDER BY TuNgay ASC;
    """,
    "FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B11_2":
    """ SELECT
        TuNgay [Ngày],
        C1 [Mã NL],
        C2 [Tên NL],
        C3 [SL Nhập Kho],
        C4 [Đ.Giá Nhập Kho],
        C5 [Thành Tiền],
        C7 [Đơn Giá PS NL Chính ],
        C8 [PS NL Chính],
        C9 [Đơn Giá PS Hương],
        C10 [PS Hương],
        C11 [Đơn Giá PS NCTT],
        C12 [PS NCTT],
        C13 [Đơn Giá PS CPC],
        C14 [PS CPC],
        C15 [Đơn Giá PS CPK],
        C16 [PS CPK]
        FROM FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B11_2
        ORDER BY TuNgay ASC;
    """,
    "FOXAI_Baocao_BangKMCP_627":
    """ SELECT
        ngay [Ngày],
        sochungtu [Số CT],
        NoiDung [Nội Dung],
        TK_NO [TK Nợ],
        TK_CO [TK Có],
        DonViTinh [ĐVT],
        SoLuong [Số Lượng],
        ThanhTien [Thành Tiền],
        Ps_ngoaite [Tiền NT],
        TyGia [Tỷ giá],
        TenNgoaiTe [Loại NT],
        DonVi [Đơn Vị]
        FROM FOXAI_Baocao_BangKMCP_627
        ORDER BY ngay ASC;
    """,
    "FOXAI_Baocao_BangKMCP_641":
    """ SELECT
        ngay [Ngày],
        sochungtu [Số CT],
        NoiDung [Nội Dung],
        TK_NO [TK Nợ],
        TK_CO [TK Có],
        DonViTinh [ĐVT],
        SoLuong [Số Lượng],
        ThanhTien [Thành Tiền],
        Ps_ngoaite [Tiền NT],
        TyGia [Tỷ giá],
        TenNgoaiTe [Loại NT],
        DonVi [Đơn Vị]
        FROM FOXAI_Baocao_BangKMCP_641
        ORDER BY ngay ASC;
    """,
    "FOXAI_Baocao_BangKMCP_642":
    """ SELECT
        ngay [Ngày],
        sochungtu [Số CT],
        NoiDung [Nội Dung],
        TK_NO [TK Nợ],
        TK_CO [TK Có],
        DonViTinh [ĐVT],
        SoLuong [Số Lượng],
        ThanhTien [Thành Tiền],
        Ps_ngoaite [Tiền NT],
        TyGia [Tỷ giá],
        TenNgoaiTe [Loại NT],
        DonVi [Đơn Vị]
        FROM FOXAI_Baocao_BangKMCP_642
        ORDER BY ngay ASC;
    """,
    "vw_ACplus_TaiKhoan":
    """ SELECT 
      TaiKhoan [Tài khoản],
	  TenTaiKhoan [Tên tài khoản]
      FROM vw_ACplus_TaiKhoan
    """
}

# Thư mục đầu ra
output_dir = r"D:\Power BI\Dự án Thuốc lá Thăng Long\Test"

for table, query in queries.items():
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    
    # Làm sạch dữ liệu: Loại bỏ dấu ngoặc đơn
    cleaned_rows = [[value[0] if isinstance(value, tuple) else value for value in row] for row in rows]
    
    # Tạo DataFrame từ kết quả đã làm sạch
    df = pd.DataFrame(cleaned_rows, columns=columns)

    # Xác định tên file cần lưu
    if table == "FOXAI_Baocao_BangCDKT":
        file_name = "F_BS.xlsx"
    elif table == "FOXAI_Baocao_KetquaKinhdoanh_PI":
        file_name = "F_IS.xlsx"
    else:
        file_name = f"{table}.xlsx"

    output_path = os.path.join(output_dir, file_name)

    # Kiểm tra và xóa file cũ nếu tồn tại
    if os.path.exists(output_path):
        os.remove(output_path)

    # Lưu DataFrame vào file Excel với xlsxwriter
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        df.to_excel(writer, index=False, sheet_name='Sheet1')
        workbook = writer.book
        worksheet = writer.sheets['Sheet1']

        # Định dạng tiêu đề cột
        header_format = workbook.add_format({
            'bold': True,
            'text_wrap': True,
            'valign': 'vcenter',
            'align': 'center',
            'fg_color': '#4F81BD',
            'font_color': '#FFFFFF'
        })
        for col_num, column in enumerate(df.columns):
            # Kiểm tra nếu cột tồn tại trong DataFrame
            if column not in df.columns:
                print(f"Cột {column} không tồn tại trong DataFrame.")
                continue
        
            column_format = workbook.add_format({'align': 'left', 'valign': 'vcenter'})
        
            # Định dạng cột "Ngày"
            if "Ngày" in column:
                column_format.set_num_format('dd/mm/yyyy')
        
            # Định dạng số cho các cột kiểu số
            elif pd.api.types.is_numeric_dtype(df[column]):
                column_format.set_align('right')
                column_format.set_num_format('#,##0.00')
        
            worksheet.set_column(col_num, col_num, 20, column_format)

    cursor.close()
    print(f"Dữ liệu từ bảng {table} đã được xuất ra file: {output_path}")

# Đóng kết nối chính
conn.close()

Dữ liệu từ bảng FOXAI_Baocao_BangCDKT đã được xuất ra file: D:\Power BI\Dự án Thuốc lá Thăng Long\Test\F_BS.xlsx
Dữ liệu từ bảng FOXAI_Baocao_KetquaKinhdoanh_PI đã được xuất ra file: D:\Power BI\Dự án Thuốc lá Thăng Long\Test\F_IS.xlsx
Dữ liệu từ bảng FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B6 đã được xuất ra file: D:\Power BI\Dự án Thuốc lá Thăng Long\Test\FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B6.xlsx
Dữ liệu từ bảng FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B12 đã được xuất ra file: D:\Power BI\Dự án Thuốc lá Thăng Long\Test\FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B12.xlsx
Dữ liệu từ bảng FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B11_2 đã được xuất ra file: D:\Power BI\Dự án Thuốc lá Thăng Long\Test\FOXAI_BaoCao_GiaThanh_DacThu_ThangLong_B11_2.xlsx
Dữ liệu từ bảng FOXAI_Baocao_BangKMCP_627 đã được xuất ra file: D:\Power BI\Dự án Thuốc lá Thăng Long\Test\FOXAI_Baocao_BangKMCP_627.xlsx
Dữ liệu từ bảng FOXAI_Baocao_BangKMCP_641 đã được xuất ra file: D:\Power BI\Dự án Thuốc lá Thăng Long\Test\F